In [ ]:
# https://www.zhihu.com/question/341222779/answer/1900671989737328833
import torch
import torch.nn as nn
import torch.nn.functional as F


class SelfAttention(nn.Module):
    def __init__(self,
                 embed_dim: int = 512,
                 num_heads: int = 8,
                 drop_rate: float = 0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads  # 有8个注意力头
        self.head_dim = self.embed_dim // self.num_heads  # 每个注意力头的维度d_k

        assert (
                self.head_dim * self.num_heads == self.embed_dim
        ), "Embedding dimension must be divisible by num_heads"

        self.W_q = nn.Linear(self.embed_dim, self.head_dim * self.num_heads, bias=False)
        self.W_k = nn.Linear(self.embed_dim, self.head_dim * self.num_heads, bias=False)
        self.W_v = nn.Linear(self.embed_dim, self.head_dim * self.num_heads, bias=False)
        self.fc = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
        self.norm = nn.LayerNorm(self.embed_dim)

    def forward(self, q: torch.Tensor, k: torch.Tensor, v: torch.Tensor, attn_mask=None) -> torch.Tensor:
        """
        q=k=v=x, x is the input features
        x.shape = (batch_size, seq_len, embed_dim)
        attn_mask.shape = (seq_len, seq_len)
        output.shape = (batch_size, seq_len, head_dim)
        """
        batch_size, seq_len, d_model = q.shape
        q = self.W_q(q) # (batch_size, seq_len, d_k*num_heads)
        q = q.view(batch_size, self.num_heads, seq_len, -1) # (batch_size, num_heads, seq_len, d_k)
        k = self.W_k(k) # (batch_size, seq_len, d_k*num_heads)
        k = k.view(batch_size, self.num_heads, seq_len, -1) # (batch_size, num_heads, seq_len, d_k)
        v = self.W_v(v) # (batch_size, seq_len, d_k*num_heads)
        v = v.view(batch_size, self.num_heads, seq_len, -1) # (batch_size, num_heads, seq_len, d_k)

        attns = torch.einsum("bhqd,bhkd->bhqk", q, k) / torch.sqrt(k.size(-1)) # (batch_size, num_heads, seq_len, seq_len)
        if attn_mask is not None:
            attns = attns.masked_fill(attn_mask == 0, float("-inf"))